In [2]:
import pandas as pd
import numpy as np
from collections import Counter
# import plotly.express as px
from collections import defaultdict

In [5]:
# Downcast for reducing storage

def downcast(data):
    data_col = data.columns.tolist()
    for col in data_col:
#         print(col)
        try:
            min_ = min(data[col].tolist())
        except:
            print('min', col)
            continue
        try:
            max_ = max(data[col].tolist())
        except:
            print('max', col)
            continue
        if 'int' in str(data[col].dtype):
            try:
                if max_ < np.iinfo('int8').max and min_ > np.iinfo('int8').min:
                    data[col] = data[col].astype('int8')
                elif max_ < np.iinfo('int16').max and min_ > np.iinfo('int16').min:
                    data[col] = data[col].astype('int16')
                elif max_ < np.iinfo('int32').max and min_ > np.iinfo('int32').min:
                    data[col] = data[col].astype('int32')
                else:
                    data[col] = data[col].astype('int64')
            except:
                print('int', col)
                pass
                
        elif 'float' in str(data[col].dtype):
            try:
                if max_ < np.finfo('float16').max and min_ > np.finfo('float16').min:
                    data[col] = data[col].astype('float16')
                elif max_ < np.finfo('float32').max and min_ > np.finfo('float32').min:
                    data[col] = data[col].astype('float32')
                else:
                    data[col] = data[col].astype('float64')
            except:
                print('float')
                pass
    return data

In [3]:
sales_train = pd.read_csv('./data/sales_train_evaluation.csv')
# sales_vali = pd.read_csv('./data/sales_train_validation.csv')
sell_price = pd.read_csv('./data/sell_prices.csv')
calendar = pd.read_csv('./data/calendar.csv')

In [ ]:
sales_train = downcast(sales_train)
# sales_vali = downcast(sales_vali)
sell_price = downcast(sell_price)
calendar = downcast(calendar)

int d_180


/Users/judytung/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py:358: FutureWarning: in the future negative indices will not be ignored by `numpy.delete`.
  self.values = np.delete(self.values, loc, 0)
/Users/judytung/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py:359: FutureWarning: in the future negative indices will not be ignored by `numpy.delete`.
  self.mgr_locs = self.mgr_locs.delete(loc)


In [ ]:
sales_train_date = pd.melt(
    sales_train,
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
#     var_name='day',
    var_name='d',
    value_name='items_sold'
)
sales_train_date = sales_train_date.merge(calendar, on='d', how='left')
sales_train_date = sales_train_date.merge(sell_price, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

In [ ]:
# sales_train_date.columns

In [ ]:
# set weekend
sales_train_date['weekend'] = (sales_train_date['wday'] <= 2).astype(int)
# set event
sales_train_date['event_name_1'] = sales_train_date['event_name_1'].fillna('No Event')
sales_train_date['event_name_2'] = sales_train_date['event_name_2'].fillna('No Event')
event_1 = sales_train_date['event_name_1'].apply(lambda x: x != 'No Event').astype(int)
event_2 = sales_train_date['event_name_1'].apply(lambda x: x != 'No Event').astype(int)
sales_train_date['event_num'] = event_1 + event_2

# fill missing sell price with mean
mean_ = sales_train_date.groupby("id")["sell_price"].transform("mean")
sales_train_date["sell_price"] = sales_train_date["sell_price"].fillna(mean_)

sales_train_date['d'] = sales_train_date['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
# snap ???

In [ ]:
sales_train_date = sales_train_date.set_index('date')

In [ ]:
cat_col = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
           'weekday', 'event_name_1', 'event_type_1', 'event_name_2',
           'event_type_2']
for col in cat_col:
    sales_train_date[col] = sales_train_date[col].astype('category')

# category -> num
for col in cat_col:
    sales_train_date[col] = sales_train_date[col].cat.codes

In [20]:
# tmp = pd.Series()
got = []

for s in sales_train['store_id']:
    if s not in got:
#         tmp = tmp.append(s)
        got.append(s)

In [26]:
pd.DataFrame({'store_id': (got)})

,store_id
0,CA_1
1,CA_2
2,CA_3
3,CA_4
4,TX_1
5,TX_2
6,TX_3
7,WI_1
8,WI_2
9,WI_3


In [27]:
pd.DataFrame({'store_id': got})['store_id'].astype('category').cat.codes

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int8

In [ ]:
sales_train_date.head()

In [ ]:
# Lag

lag = [1, 2, 7, 14, 28, 30, 60, 90]
for l in lag:
    sales_train_date['day_lag_{}'.format(l)] = sales_train_date.groupby('id')["items_sold"].shift(l).astype(np.float16)
    
sales_train_date = sales_train_date[sales_train_date['day_lag_90'].notnull()].reset_index()


In [ ]:
drop_col = ['id', 'cat_id', 'state_id', 'd', 'wm_yr_wk', 'weekday', 'wday',
            'month', 'year', 'event_name_1','event_type_1', 'event_name_2',
            'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']
sales_train_date = sales_train_date.drop(drop_col, axis=1)


In [ ]:
sales_train_date.columns

In [ ]:
sales_train_date.shape

In [ ]:
sales_train_date.head()

In [29]:
sales_train_date.to_csv('./data/sales_train_date.csv')

In [20]:
sales_train_date

,date,item_id,dept_id,store_id,sell_price,weekend,event_num,day_lag_1,day_lag_2,day_lag_7,day_lag_14,day_lag_28,day_lag_30,day_lag_60,day_lag_90
0,2011-04-29,1437,3,0,8.281250,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2011-04-29,1438,3,0,3.970703,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011-04-29,1439,3,0,2.970703,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011-04-29,1440,3,0,4.339844,0,0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
4,2011-04-29,1441,3,0,2.943359,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2011-04-29,1442,3,0,0.990723,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2011-04-29,1443,3,0,7.882812,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2011-04-29,1444,3,0,0.500000,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.0
8,2011-04-29,1445,3,0,1.769531,0,0,3.0,3.0,2.0,1.0,0.0,0.0,0.0,2.0
9,2011-04-29,1446,3,0,3.169922,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
